In [2]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itdt20" #TODO
fs_method, fs_ratio = "dnp", 0.5  #dnp & graces
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [06:07<00:00,  5.17s/it]


In [3]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 1/50:  46%|████▌     | 12/26 [00:00<00:00, 118.01batch/s, loss=0.29]

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9145 | AUPR: 0.6307 | Acc: 95.19%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9185 | AUPR: 0.6430 | Acc: 95.24%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9201 | AUPR: 0.6507 | Acc: 95.11%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9163 | AUPR: 0.6383 | Acc: 95.22%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9219 | AUPR: 0.6541 | Acc: 95.33%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9178 | AUPR: 0.6497 | Acc: 95.30%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9219 | AUPR: 0.6541 | Acc: 95.33%
Internal test: AUC: 0.9241 | AUPR: 0.6718 | Acc: 94.91%
External test: AUC: 0.9157 | AUPR: 0.6428 | Acc: 94.96%


In [4]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9158 | AUPR: 0.6126 | Acc: 94.91%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9168 | AUPR: 0.6121 | Acc: 94.82%
model(kernel=linear, C=1)
Valid:         AUC: 0.9164 | AUPR: 0.6116 | Acc: 94.84%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9168 | AUPR: 0.6121 | Acc: 94.82%
Internal test: AUC: 0.9186 | AUPR: 0.6453 | Acc: 94.72%
External test: AUC: 0.9103 | AUPR: 0.5910 | Acc: 94.38%


In [5]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7521 | AUPR: 0.4230 | Acc: 94.70%
model(n_neighbors=5)
Valid:         AUC: 0.7839 | AUPR: 0.4676 | Acc: 94.94%
model(n_neighbors=7)
Valid:         AUC: 0.7985 | AUPR: 0.4872 | Acc: 94.98%
best model(n_neighbors=7)
Valid:         AUC: 0.7985 | AUPR: 0.4872 | Acc: 94.98%
Internal test: AUC: 0.8066 | AUPR: 0.5200 | Acc: 94.79%
External test: AUC: 0.8049 | AUPR: 0.4965 | Acc: 94.78%


In [6]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8628 | AUPR: 0.3759 | Acc: 83.18%
best model()
Valid:         AUC: 0.8628 | AUPR: 0.3759 | Acc: 83.18%
Internal test: AUC: 0.8618 | AUPR: 0.3928 | Acc: 82.41%
External test: AUC: 0.8536 | AUPR: 0.3548 | Acc: 81.71%


In [7]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9172 | AUPR: 0.6360 | Acc: 95.12%
model(C=10.0)
Valid:         AUC: 0.9169 | AUPR: 0.6351 | Acc: 95.13%
model(C=100.0)
Valid:         AUC: 0.9168 | AUPR: 0.6350 | Acc: 95.12%
best model(C=1.0)
Valid:         AUC: 0.9172 | AUPR: 0.6360 | Acc: 95.12%
Internal test: AUC: 0.9199 | AUPR: 0.6638 | Acc: 94.94%
External test: AUC: 0.9103 | AUPR: 0.6152 | Acc: 94.69%


In [8]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8634 | AUPR: 0.5501 | Acc: 80.79%
model(criterion=log_loss)
Valid:         AUC: 0.8755 | AUPR: 0.5696 | Acc: 80.42%
model(criterion=entropy)
Valid:         AUC: 0.8755 | AUPR: 0.5696 | Acc: 80.42%
best model(criterion=log_loss)
Valid:         AUC: 0.8755 | AUPR: 0.5696 | Acc: 80.42%
Internal test: AUC: 0.8703 | AUPR: 0.5992 | Acc: 84.36%
External test: AUC: 0.8690 | AUPR: 0.5768 | Acc: 83.54%


In [9]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier


model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9092 | AUPR: 0.6452 | Acc: 88.07%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9089 | AUPR: 0.6428 | Acc: 88.30%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9092 | AUPR: 0.6436 | Acc: 88.34%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9077 | AUPR: 0.6447 | Acc: 88.04%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9091 | AUPR: 0.6446 | Acc: 88.50%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9095 | AUPR: 0.6441 | Acc: 88.58%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9077 | AUPR: 0.6447 | Acc: 88.04%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9091 | AUPR: 0.6446 | Acc: 88.50%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9095 | AUPR: 0.6441 | Acc: 88.58%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9095 | AUPR: 0.6441 | Acc: 88.58%
Internal test

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9179 | AUPR: 0.6536 | Acc: 95.27%
model(criterion=squared_error)
Valid:         AUC: 0.9186 | AUPR: 0.6562 | Acc: 95.27%
best model(criterion=squared_error)
Valid:         AUC: 0.9186 | AUPR: 0.6562 | Acc: 95.27%
Internal test: AUC: 0.9249 | AUPR: 0.6907 | Acc: 95.06%
External test: AUC: 0.9184 | AUPR: 0.6581 | Acc: 95.17%


In [11]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9138 | AUPR: 0.6203 | Acc: 95.00%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9139 | AUPR: 0.6163 | Acc: 94.97%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9132 | AUPR: 0.6142 | Acc: 94.90%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9139 | AUPR: 0.6163 | Acc: 94.97%
Internal test: AUC: 0.9140 | AUPR: 0.6498 | Acc: 94.96%
External test: AUC: 0.9102 | AUPR: 0.6005 | Acc: 94.59%


In [12]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)